# Results Components

> UI components for displaying transcription results and errors

In [ ]:
#| default_exp components.results

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from fasthtml.common import *
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_sizes, btn_styles
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title, card_actions
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, gap, flex_wrap
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_workflow_transcription_single_file.core.config import SingleFileWorkflowConfig
from cjm_fasthtml_workflow_transcription_single_file.core.html_ids import SingleFileHtmlIds

## transcription_results

Displays completed transcription results with transcript text, metadata, and action buttons.

In [ ]:
#| export
def transcription_results(
    job_id: str, # Unique identifier for the transcription job
    transcription_text: str, # The transcribed text
    metadata: Dict[str, Any], # Transcription metadata from the plugin
    file_info: Dict[str, Any], # Dictionary with file details (name, path, type, size_str)
    plugin_info: Dict[str, Any], # Dictionary with plugin details (id, title, supports_streaming)
    config: SingleFileWorkflowConfig, # Workflow configuration
    router: APIRouter, # Workflow router for generating route URLs
    stepflow_router: APIRouter, # StepFlow router for generating stepflow URLs
) -> FT: # FastHTML component showing results with export options
    """Render transcription results with export options."""
    # Calculate word/character counts
    word_count = len(transcription_text.split())
    char_count = len(transcription_text)

    # Build URLs using router's .to() method for proper route generation
    new_transcription_url = stepflow_router.start.to()
    export_formats = config.export_formats

    return Div(
        Div(
            H2("Transcription Complete", cls=str(card_title)),

            Div(
                # Summary info
                Div(
                    Div(
                        P(f"File: {file_info.get('name', 'Unknown')}",
                          cls=combine_classes(font_size.sm)),
                        P(f"Plugin: {plugin_info.get('title', 'Unknown')}",
                          cls=combine_classes(font_size.sm)),
                        P(f"Words: {word_count} | Characters: {char_count}",
                          cls=combine_classes(font_size.sm)),
                        cls=combine_classes(text_dui.base_content.opacity(80), m.b(4))
                    )
                ),

                # Transcription text area
                Div(
                    H3("Transcription Text",
                       cls=combine_classes(font_weight.semibold, m.b(2))),
                    Textarea(
                        transcription_text,
                        id=SingleFileHtmlIds.RESULTS_TEXT,
                        rows="15",
                        readonly=True,
                        cls=combine_classes(
                            w.full,
                            p(3),
                            bg_dui.base_200,
                            border(),
                            border_dui.base_300,
                            border_radius.box,
                            font_size.sm,
                        )
                    ),
                    cls=str(m.b(6))
                ),

                # Export options
                Div(
                    H3("Export Options",
                       cls=combine_classes(font_weight.semibold, m.b(3))),
                    Div(
                        *[
                            A(
                                f"Download as {fmt.upper()}",
                                href=router.export.to(job_id=job_id, format=fmt),
                                cls=combine_classes(btn, btn_sizes.sm, btn_styles.ghost)
                            )
                            for fmt in export_formats
                        ],
                        Button(
                            "Copy to Clipboard",
                            onclick=f"""
                                navigator.clipboard.writeText(document.getElementById('{SingleFileHtmlIds.RESULTS_TEXT}').value)
                                .then(() => {{
                                    this.textContent = 'Copied!';
                                    setTimeout(() => {{ this.textContent = 'Copy to Clipboard'; }}, 2000);
                                }});
                            """,
                            cls=combine_classes(btn, btn_sizes.sm, btn_colors.primary, m.l(2)),
                            id=SingleFileHtmlIds.COPY_BTN
                        ),
                        cls=combine_classes(flex_display, gap(2), flex_wrap.wrap)
                    ),
                    cls=str(m.b(6)),
                    id=SingleFileHtmlIds.RESULTS_EXPORT
                ),

                # Actions
                Div(
                    Button(
                        "New Transcription",
                        hx_get=new_transcription_url,
                        hx_target=SingleFileHtmlIds.as_selector(config.container_id),
                        hx_swap="innerHTML",
                        cls=combine_classes(btn, btn_colors.primary),
                        id=SingleFileHtmlIds.NEW_TRANSCRIPTION_BTN
                    ),
                    cls=str(card_actions)
                ),

                cls=str(card_body)
            ),
            cls=combine_classes(card, bg_dui.base_100)
        ),
        # Use workflow's container_id so HTMX outerHTML swap preserves the target
        id=config.container_id
    )

## transcription_error

Displays transcription error state with error message and retry option.

In [ ]:
#| export
def transcription_error(
    error_message: str, # Description of the error that occurred
    file_info: Optional[Dict[str, Any]], # Optional dictionary with file details
    config: SingleFileWorkflowConfig, # Workflow configuration
    stepflow_router: APIRouter, # StepFlow router for generating stepflow URLs
) -> FT: # FastHTML component showing error with retry option
    """Render transcription error message."""
    # Build URL using router's .to() method for proper route generation
    new_transcription_url = stepflow_router.start.to()

    return Div(
        Div(
            H2("Transcription Failed", cls=str(card_title)),

            Div(
                Div(
                    Span("Error", cls=combine_classes(badge, badge_colors.error, m.b(3))),
                    P(error_message,
                      cls=combine_classes(text_dui.error, m.b(4))),

                    P(f"File: {file_info.get('name', 'Unknown')}",
                      cls=combine_classes(font_size.sm, text_dui.base_content.opacity(70)))
                    if file_info else None,
                    cls=str(m.b(6))
                ),

                Div(
                    Button(
                        "Try Again",
                        hx_get=new_transcription_url,
                        hx_target=SingleFileHtmlIds.as_selector(config.container_id),
                        hx_swap="innerHTML",
                        cls=combine_classes(btn, btn_colors.primary),
                        id=SingleFileHtmlIds.NEW_TRANSCRIPTION_BTN
                    ),
                    cls=str(card_actions)
                ),

                cls=str(card_body)
            ),
            cls=combine_classes(card, bg_dui.base_100)
        ),
        # Use workflow's container_id so HTMX outerHTML swap preserves the target
        id=config.container_id
    )

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()